# **TAREA LanceDB**
- Considera usar ANN para cada búsqueda o filtro

**Task 1: Consulta avanzada con proyección y filtro**

Instrucciones:
1. Genera un vector aleatorio con la misma dirección que los embeddings que están en la tabla mis_vectores
2. Realiza una búsqueda en la tabla para encontrar los 5 elementos más cercanos
3. Proyecta los resultados para mostrar solo las columnas item y _distance
4. Excluye de los resultado los elementos cuyo nombre sea 'item 500'

Pregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?

In [1]:
# Cargar librerías necesarias
import time
import lancedb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Configurar LanceDB
uri = "data/ANN"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# Crear 10,000 vectores de muestra aleatorios
np.random.seed(42)  # Para reproducibilidad
data = [
    {"vector": row, "item": f"item {i}"}
    for i, row in enumerate(np.random.random((19_999_0, 1536)).astype("float32"))
]

# Crear tabla en la base de datos
tbl = db.create_table("mis_vectores", data=data)
print(f"Tabla 'mis_vectores' creada con {len(data)} vectores.")
# Mostrar las primeras 5 filas de la tabla

Tabla 'mis_vectores' creada con 199990 vectores.


In [3]:
query_vector = np.random.random(1536).astype("float32")

results = tbl.search(query_vector).limit(5).to_pandas()

filtered_results = results[results["item"] != "item 500"]

final_results = filtered_results[["item", "_distance"]]

print(final_results)


          item   _distance
0  item 120138  226.667191
1   item 26019  227.885849
2   item 17350  228.319977
3   item 30532  228.503052
4  item 121832  229.366043


**Task 2: Creación de tablas**

Instrucciones:
1. Define un nuevo esquema para una tabla vacía con las siguientes columnas:
*   vector (vector de 4 dimensiones)
*   nombre
*   categoria
2. Crea una tabla vacía llamada nueva_tabla usando el esquema
3. Inserta 5 registros en la tabla
4. Muestra el contenido de la tabla





In [11]:
import pyarrow as pa
import lancedb

# Paso 1: Definir el esquema usando PyArrow
schema = pa.schema([
    ("vector", pa.list_(pa.float32(), 4)),  # Vector de 4 dimensiones
    ("nombre", pa.string()),               # Columna de tipo string
    ("categoria", pa.string()),            # Columna de tipo string
])

# Paso 2: Crear la base de datos
db = lancedb.connect("data/ANN")

# Paso 3: Crear o sobrescribir la tabla llamada "nueva_tabla"
nueva_tabla = db.create_table("nueva_tabla", schema=schema, mode="overwrite")

# Paso 4: Insertar registros como una lista de diccionarios
records = [
    {"vector": [0.1, 0.2, 0.3, 0.4], "nombre": "Registro 1", "categoria": "A"},
    {"vector": [0.5, 0.6, 0.7, 0.8], "nombre": "Registro 2", "categoria": "B"},
    {"vector": [0.9, 1.0, 1.1, 1.2], "nombre": "Registro 3", "categoria": "A"},
    {"vector": [1.3, 1.4, 1.5, 1.6], "nombre": "Registro 4", "categoria": "C"},
    {"vector": [1.7, 1.8, 1.9, 2.0], "nombre": "Registro 5", "categoria": "B"},
]

nueva_tabla.add(records)

# Paso 5: Mostrar el contenido de la tabla
print(nueva_tabla.to_pandas())


                 vector      nombre categoria
0  [0.1, 0.2, 0.3, 0.4]  Registro 1         A
1  [0.5, 0.6, 0.7, 0.8]  Registro 2         B
2  [0.9, 1.0, 1.1, 1.2]  Registro 3         A
3  [1.3, 1.4, 1.5, 1.6]  Registro 4         C
4  [1.7, 1.8, 1.9, 2.0]  Registro 5         B


**Task 3: Actualización de vectores y filtrado**

Instrucciones:
1. Crea una tabla utilizando un DataFrame de Pandas con las siguientes columnas:
- id (Entero).
- vector (Lista de tres números flotantes)
2. Actualiza el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
3. Filtra la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0


In [13]:
import pandas as pd
import lancedb

data = {
    "id": [1, 2, 3, 4, 5],
    "vector": [[0.1, 0.2, 0.3], [1.1, 1.2, 1.3], [2.1, 2.2, 2.3], [3.1, 3.2, 3.3], [4.1, 4.2, 4.3]],
}
df = pd.DataFrame(data)

db = lancedb.connect("data/ANN")
tabla = db.create_table("tabla_actualizacion", data=df, mode="overwrite")

tabla.update("id == 3", {"vector": [10.0, 11.0, 10.0]})

resultados = tabla.to_pandas()
resultados_filtrados = resultados[resultados["vector"].apply(lambda x: any(val > 9.0 for val in x))]

print(resultados_filtrados)


   id              vector
4   3  [10.0, 11.0, 10.0]


**Task 4: Embeddings multimodales y búsqueda combinada**

Instrucciones:
1. Crea una tabla con datos de texto e imágenes combinados. Incluye las siguientes columnas:
- texto (Texto).
- imagen (Nombre del archivo de imagen).
- embedding_texto (Vector del texto generado con SentenceTransformer).
- embedding_imagen (Vector de la imagen generado con ResNet18).
2. Realiza una consulta para encontrar los elementos con un texto similar a "La tecnología avanza rápido" y una imagen visualmente similar a un color predominantemente azul.
3. Muestra los resultados combinados ordenados por la menor distancia promedio entre ambos embeddings.


In [14]:
import os
from sentence_transformers import SentenceTransformer
import torch
from torchvision import models, transforms
from PIL import Image
import lancedb
import numpy as np
import pandas as pd

modelo_texto = SentenceTransformer('all-MiniLM-L6-v2')
modelo_imagen = models.resnet18(pretrained=True)
modelo_imagen.fc = torch.nn.Identity()  # Remover la capa final para obtener embeddings

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

ruta_imagenes = "./data/images"
archivos_imagenes = [f for f in os.listdir(ruta_imagenes) if f.endswith('.jpg')]

textos = [
    "El cielo es azul",
    "La tecnología avanza rápido",
    "Un día soleado",
    "Innovación y progreso",
    "El océano azul profundo",
    "Montañas bajo el cielo claro"
]

data = []
for texto, archivo_imagen in zip(textos, archivos_imagenes):
    # Embedding del texto
    embedding_texto = modelo_texto.encode(texto)
    
    # Embedding de la imagen
    imagen = Image.open(os.path.join(ruta_imagenes, archivo_imagen)).convert("RGB")
    tensor_imagen = transform(imagen).unsqueeze(0)  # Añadir batch dimension
    embedding_imagen = modelo_imagen(tensor_imagen).detach().numpy().flatten()
    
    # Agregar datos a la tabla
    data.append({
        "texto": texto,
        "imagen": archivo_imagen,
        "embedding_texto": embedding_texto,
        "embedding_imagen": embedding_imagen
    })

df = pd.DataFrame(data)

db = lancedb.connect("data/ANN")
tabla = db.create_table("tabla_multimodal", data=df.to_dict(orient="records"), mode="overwrite")

consulta_texto = "La tecnología avanza rápido"
consulta_imagen = np.random.rand(512)  # Simular un embedding para un color azul predominante (ajusta según tu necesidad)

resultados = []
for _, fila in df.iterrows():
    dist_texto = np.linalg.norm(fila["embedding_texto"] - modelo_texto.encode(consulta_texto))
    dist_imagen = np.linalg.norm(fila["embedding_imagen"] - consulta_imagen)
    dist_promedio = (dist_texto + dist_imagen) / 2
    resultados.append({
        "texto": fila["texto"],
        "imagen": fila["imagen"],
        "distancia_promedio": dist_promedio
    })

resultados_ordenados = sorted(resultados, key=lambda x: x["distancia_promedio"])

print(pd.DataFrame(resultados_ordenados).head())


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth

[2024-12-05T04:01:15Z WARN  lance::dataset] No existing dataset at /usr/src/app/notebooks/data/ANN/tabla_multimodal.lance, it will be created


                          texto        imagen  distancia_promedio
0   La tecnología avanza rápido  imagen_2.jpg            5.828258
1                Un día soleado  imagen_3.jpg            6.410706
2       El océano azul profundo  imagen_5.jpg            6.445819
3  Montañas bajo el cielo claro  imagen_6.jpg            6.455087
4              El cielo es azul  imagen_1.jpg            6.488733
